# 990PF 

In [1]:
import pandas as pd
from xml.etree import cElementTree as ElementTree
import xml.etree.ElementTree as et
from tqdm import tqdm
import logging
import re

logging.basicConfig(format='%(asctime)s: %(message)s', filename='analysis.log', level=logging.DEBUG)

In [2]:
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

def get_field(xmldict, path=None, default=None):
    if path is None: return xmldict
    try:
        for p in path:
            # Ugly namespaces are ugly, let's deal with them here
            xmldict = xmldict['{http://www.irs.gov/efile}' + p]
        return xmldict
    except KeyError:
        # If this key does not exist in the dictionary, 
        return default
        # Note: it may be faster to check ```if p not in dict``` rather
        # than set up this try/except block. I think it depends on how
        # often this fails - the more it fails, the worse the try/except does.

In [45]:
tree = et.parse("data/test/201721229349200737_public.xml")
root = tree.getroot()
#Converting from xml to dictionary
xmldict = XmlDictConfig(root)

get_field(xmldict, ['ReturnData','IRS990ScheduleG','FundraisingEventInformationGrp', 'GrossRevenueTotalEventsAmt'])

'63814'

In [3]:
def analyze_one(path):
    object_id = re.sub("\D", "", path)
    object_list = [[object_id]]
    object_df = pd.DataFrame(object_list, columns =['Object_ID'])
    tree = et.parse(path)
    root = tree.getroot()
    #Converting from xml to dictionary
    xmldict = XmlDictConfig(root)

    
    ##############Header Information 
    #Date Information
    TaxPeriodBeginDt = get_field(xmldict, ['ReturnHeader','TaxPeriodBeginDt'])
    TaxPeriodEndDt = get_field(xmldict, ['ReturnHeader','TaxPeriodEndDt'])
    
    
    #Filer Information
    filer_data = get_field(xmldict, ['ReturnHeader', 'Filer'], {})
    EIN = get_field(filer_data, ['EIN'])
    BusinessName = get_field(filer_data, ['BusinessName', 'BusinessNameLine1Txt'])
    City = get_field(filer_data, ['USAddress', 'CityNm'])
    State = get_field(filer_data, ['USAddress', 'StateAbbreviationCd'])
    ZIPCd = get_field(filer_data, ['USAddress', 'ZIPCd'])
    TaxYr = get_field(xmldict, ['ReturnHeader', 'TaxYr'])
    filer_list = [[EIN, BusinessName,TaxPeriodBeginDt, TaxPeriodEndDt,TaxYr, City, State, ZIPCd]]
    filer_df = pd.DataFrame(filer_list, columns = ['EIN', 'BusinessName','TaxPeriodBeginDt','TaxPeriodEndDt','TaxYr','City', 'State', 'ZIPCd']) 
    
    
    ##############Return Data
    return_data = get_field(xmldict, ['ReturnData', 'IRS990EZ'])
    if return_data is None:
        logging.warning(f'Could not find Return Data for file: {path}')
        return None
    
    #AddressChangeInd
    AddressChangeInd = get_field(return_data, ['AddressChangeInd'], "")
    
    #InitialReturnInd
    InitialReturnInd = get_field(return_data, ['InitialReturnInd'], "")
    
    #InitialReturnFormerPubChrtyInd
    InitialReturnFormerPubChrtyInd = get_field(return_data, ['InitialReturnFormerPubChrtyInd'], "")
    
    #AmendedReturnInd
    AmendedReturnInd = get_field(return_data, ['AmendedReturnInd'], "")
    
    #FinalReturnInd
    FinalReturnInd = get_field(return_data, ['FinalReturnInd'], "")
    
    #NameChange this may only be for paper filers 
    NameChange = get_field(return_data, ['NameChange'], "")
    
    #Organization501c3Ind
    Organization501c3Ind = get_field(return_data, ['Organization501c3Ind'], "")
    
    #Organization501cInd
    Organization501cInd = get_field(return_data, ['Organization501cInd'], "")
    
    #Organization4947a1NotPFInd
    Organization4947a1NotPFInd = get_field(return_data, ['Organization4947a1NotPFInd'], "")
    
    #Organization527Ind
    Organization527Ind = get_field(return_data, ['Organization527Ind'], "")
    
    #TypeOfOrganizationCorpInd
    TypeOfOrganizationCorpInd = get_field(return_data, ['TypeOfOrganizationCorpInd'], "")
    
    #Trust
    TypeOfOrganizationTrustInd = get_field(return_data, ['TypeOfOrganizationTrustInd'], "")
    
    #asso
    TypeOfOrganizationAssocInd = get_field(return_data, ['TypeOfOrganizationAssocInd'], "")
    
    #other
    TypeOfOrganizationOtherInd = get_field(return_data, ['TypeOfOrganizationOtherInd'], "")
    
    #GrossReceiptsAmt
    GrossReceiptsAmt = get_field(return_data, ['GrossReceiptsAmt'], "0")
    
    #InfoInScheduleOPartIInd
    InfoInScheduleOPartIInd = get_field(return_data, ['InfoInScheduleOPartIInd'], "")
    
    #ContributionsGiftsGrantsEtcAmt
    ContributionsGiftsGrantsEtcAmt = get_field(return_data, ['ContributionsGiftsGrantsEtcAmt'], "0")
    
    #MembershipDuesAmt
    ProgramServiceRevenueAmt = get_field(return_data, ['ProgramServiceRevenueAmt'], "0")
    
    #GrossReceiptsAmt
    MembershipDuesAmt = get_field(return_data, ['MembershipDuesAmt'], "0")
    
    #InvestmentIncomeAmt
    InvestmentIncomeAmt = get_field(return_data, ['InvestmentIncomeAmt'], "0")
    
    #GamingGrossIncomeAmt
    GamingGrossIncomeAmt = get_field(return_data, ['GamingGrossIncomeAmt'], "0")
    
    #FundraisingGrossIncomeAmt
    FundraisingGrossIncomeAmt = get_field(return_data, ['FundraisingGrossIncomeAmt'], "0")
    
    #Fundraising from scheduleG
    GrossRevenueTotalEventsAmt= get_field(xmldict, ['ReturnData','IRS990ScheduleG','FundraisingEventInformationGrp', 'GrossRevenueTotalEventsAmt'])
    
    # TotalRevenueAmt
    TotalRevenueAmt = get_field(return_data, ['TotalRevenueAmt'], "0")
    
    #TotalExpensesAmtt
    TotalExpensesAmt = get_field(return_data, ['TotalExpensesAmt'], "0")
    
    #NetAssetsOrFundBalancesBOYAmt
    NetAssetsOrFundBalancesBOYAmt = get_field(return_data, ['NetAssetsOrFundBalancesBOYAmt'], "0")
    
    #NetAssetsOrFundBalancesEOYAmt
    NetAssetsOrFundBalancesEOYAmt = get_field(return_data, ['NetAssetsOrFundBalancesEOYAmt'], "0")
    
    #Form990TotalAssetsGrp
    Form990TotalAssetsGrpBOY = get_field(return_data, ['Form990TotalAssetsGrp', 'BOYAmt'], "0")
    Form990TotalAssetsGrpEOY = get_field(return_data, ['Form990TotalAssetsGrp', 'EOYAmt'], "0")
    
    #SumOfTotalLiabilitiesGrp
    SumOfTotalLiabilitiesGrpBOY = get_field(return_data, ['SumOfTotalLiabilitiesGrp','BOYAmt'], "0")
    SumOfTotalLiabilitiesGrpEOY = get_field(return_data, ['SumOfTotalLiabilitiesGrp','EOYAmt'], "0")
    
    #NetAssetsOrFundBalancesGrp
    NetAssetsOrFundBalancesGrpBOY = get_field(return_data, ['NetAssetsOrFundBalancesGrp','BOYAmt'], "0")
    NetAssetsOrFundBalancesGrpEOY = get_field(return_data, ['NetAssetsOrFundBalancesGrp','EOYAmt'], "0")
    
    #OperateHospitalInd
    OperateHospitalInd = get_field(return_data, ['OperateHospitalInd'], "")
    
    #SchoolOperatingInd
    SchoolOperatingInd = get_field(return_data, ['SchoolOperatingInd'], "")
    

    return_list = [[    AddressChangeInd,
                        InitialReturnInd,
                        InitialReturnFormerPubChrtyInd,
                        AmendedReturnInd,
                        FinalReturnInd,
                        NameChange,
                        Organization501c3Ind,
                        Organization501cInd,
                        Organization4947a1NotPFInd,
                        Organization527Ind,
                        TypeOfOrganizationCorpInd,
                        TypeOfOrganizationTrustInd,
                        TypeOfOrganizationAssocInd,
                        TypeOfOrganizationOtherInd,
                        GrossReceiptsAmt,
                        InfoInScheduleOPartIInd,
                        ContributionsGiftsGrantsEtcAmt,
                        MembershipDuesAmt,
                        GrossReceiptsAmt,
                        InvestmentIncomeAmt,
                        GamingGrossIncomeAmt,
                        FundraisingGrossIncomeAmt,
                        GrossRevenueTotalEventsAmt,
                        TotalRevenueAmt,
                        TotalExpensesAmt,
                        NetAssetsOrFundBalancesBOYAmt,
                        NetAssetsOrFundBalancesEOYAmt,
                        Form990TotalAssetsGrpBOY,
                        Form990TotalAssetsGrpEOY,
                        SumOfTotalLiabilitiesGrpBOY,
                        SumOfTotalLiabilitiesGrpEOY,
                        NetAssetsOrFundBalancesGrpBOY,
                        NetAssetsOrFundBalancesGrpEOY,
                        OperateHospitalInd,
                        SchoolOperatingInd
                        
                   ]]
    
    return_df = pd.DataFrame(return_list, columns =
                             ['AddressChangeInd',
                            'InitialReturnInd',
                            'InitialReturnFormerPubChrtyInd',
                            'AmendedReturnInd',
                            'FinalReturnInd',
                            'NameChange',
                            'Organization501c3Ind',
                            'Organization501cInd',
                            'Organization4947a1NotPFInd',
                            'Organization527Ind',
                            'TypeOfOrganizationCorpInd',
                            'TypeOfOrganizationTrustInd',
                            'TypeOfOrganizationAssocInd',
                            'TypeOfOrganizationOtherInd',
                            'GrossReceiptsAmt',
                            'InfoInScheduleOPartIInd',
                            'ContributionsGiftsGrantsEtcAmt',
                            'MembershipDuesAmt',
                            'GrossReceiptsAmt',
                            'InvestmentIncomeAmt',
                            'GamingGrossIncomeAmt',
                            'FundraisingGrossIncomeAmt',
                            'GrossRevenueTotalEventsAmt',
                            'TotalRevenueAmt',
                            'TotalExpensesAmt',
                            'NetAssetsOrFundBalancesBOYAmt',
                            'NetAssetsOrFundBalancesEOYAmt',
                            'Form990TotalAssetsGrpBOY',
                            'Form990TotalAssetsGrpEOY',
                            'SumOfTotalLiabilitiesGrpBOY',
                            'SumOfTotalLiabilitiesGrpEOY',
                            'NetAssetsOrFundBalancesGrpBOY',
                            'NetAssetsOrFundBalancesGrpEOY',
                             'OperateHospitalInd',
                             'SchoolOperatingInd'])

    #Concatenating header information and contributions
    return pd.concat([object_df, filer_df, return_df], axis=1, ignore_index=False)
    

In [4]:
#This is the method I used to get all the files in the directory

import os

'''
    For the given path, get the List of all files in the directory tree 
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles


dirName = 'C:/Users/Administrator/Desktop/irs_2019';
files = getListOfFiles(dirName)

In [5]:

df = pd.concat([one_row for name in tqdm(files) if (one_row := analyze_one(name)) is not None])
df

100%|██████████| 475575/475575 [43:45<00:00, 181.13it/s] 


,Object_ID,EIN,BusinessName,TaxPeriodBeginDt,TaxPeriodEndDt,TaxYr,City,State,ZIPCd,AddressChangeInd,...,NetAssetsOrFundBalancesBOYAmt,NetAssetsOrFundBalancesEOYAmt,Form990TotalAssetsGrpBOY,Form990TotalAssetsGrpEOY,SumOfTotalLiabilitiesGrpBOY,SumOfTotalLiabilitiesGrpEOY,NetAssetsOrFundBalancesGrpBOY,NetAssetsOrFundBalancesGrpEOY,OperateHospitalInd,SchoolOperatingInd
0,2019201900079349200000,431533247,SENIOR FOUNDATION OF THE OZARKS,2017-07-01,2018-06-30,2017,SPRINGFIELD,MO,65806,X,...,273189,266054,290816,281458,17627,15404,273189,266054,false,false
0,2019201900079349200005,953416505,BLAZERS YOUTH SERVICES COMMUNITY CLUB INC,2017-03-01,2018-02-28,2017,Los Angeles,CA,90062,,...,56948,57117,56948,57117,0,0,56948,57117,false,false
0,2019201900079349200010,200028509,MCKINLEY PARENT TEACHER CLUB TRUST,2017-09-01,2018-08-31,2017,SALEM,OR,97302,,...,24877,31437,24877,31437,0,0,24877,31437,false,false
0,2019201900079349200050,510195911,PI BETA PHI FRATERNITY TEXAS EPSILON CHAPTER,2017-07-01,2018-06-30,2017,DENTON,TX,76202,,...,302480,313375,308480,319375,6000,6000,302480,313375,false,
0,2019201900079349200055,208006058,ARCHBOLD FISH,2017-10-01,2018-09-30,2017,Archbold,OH,43502,,...,61731,59677,62363,60066,632,389,61731,59677,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019201943579349200759,391463454,SIREN CHAMBER OF COMMERCE INC,2018-09-01,2019-08-31,2018,SIREN,WI,54872,,...,20159,12166,20435,12166,276,0,20159,12166,false,
0,2019201943579349200764,810526413,Bozeman Figure Skating Club,2018-09-01,2019-08-31,2018,Bozeman,MT,59772,,...,10140,21159,10339,21215,199,56,10140,21159,false,false
0,2019201943609349200004,411690310,STILLWATER AREA,2018-09-01,2019-08-31,2018,WOODBURY,MN,55129,,...,50918,69624,50918,69624,0,0,50918,69624,false,false
0,2019201943609349200104,452475225,THE NORTH UNION ATHLETIC,2018-07-01,2019-06-30,2018,RICHWOOD,OH,43344,,...,61999,75793,61999,75793,0,0,61999,75793,false,false


In [6]:
df.to_csv("C:/Users/Administrator/Desktop/Impact_Capital/data/ez/2019_objects.csv")